# Preprocessing

Run a script found from the internet. Has a lot of useful regex, and we're comandeering some of it's functions for our own use.

In [1]:
%run ../src/twitter-sentiment/preprocessing/__init__.py

import pandas as pd


In [3]:
f  = '../data/external/classifier/twitter-hate-speech-classifier.csv'
keep_cols = ['does_this_tweet_contain_hate_speech', 'tweet_text']
df = pd.read_csv(f, usecols=keep_cols)

df.head(20)

,does_this_tweet_contain_hate_speech,tweet_text
0,The tweet uses offensive language but not hate...,Warning: penny boards will make you a faggot
1,The tweet contains hate speech,Fuck dykes
2,The tweet contains hate speech,@sizzurp__ @ILIKECATS74 @yoPapi_chulo @brandon...
3,The tweet contains hate speech,"""@jayswaggkillah: ""@JacklynAnnn: @jayswaggkill..."
4,The tweet uses offensive language but not hate...,@Zhugstubble You heard me bitch but any way I'...
5,The tweet contains hate speech,@elaynay your a dirty terrorist and your relig...
6,The tweet contains hate speech,RT @ivanrabago_: @_WhitePonyJr_ looking like f...
7,The tweet contains hate speech,Well I thought you knew actually RT @KingHorse...
8,The tweet uses offensive language but not hate...,"@Stonisnipezz I know. It was a joke, faggot."
9,The tweet uses offensive language but not hate...,I'm tired of people saying I look like my brot...


### Pipeline

**Label mapping**

In [5]:
# remap labels for ML 
label_map = {
    'The tweet is not offensive': 0,
    'The tweet uses offensive language but not hate speech': 1,
    'The tweet contains hate speech': 2
}
def map_label(label):
    return label_map[label]

# map labels
#df['label'] = df.does_this_tweet_contain_hate_speech.apply(lambda x: label_map[x])
df['label'] = df.does_this_tweet_contain_hate_speech.map(map_label)

#df['label2'] = df.label.apply(lambda x: 1 if (x==2) else 0)

In [7]:
df.head()

,does_this_tweet_contain_hate_speech,tweet_text,label
0,The tweet uses offensive language but not hate...,Warning: penny boards will make you a faggot,1
1,The tweet contains hate speech,Fuck dykes,2
2,The tweet contains hate speech,@sizzurp__ @ILIKECATS74 @yoPapi_chulo @brandon...,2
3,The tweet contains hate speech,"""@jayswaggkillah: ""@JacklynAnnn: @jayswaggkill...",2
4,The tweet uses offensive language but not hate...,@Zhugstubble You heard me bitch but any way I'...,1


In [4]:
# tokenize
df['tweet_clean'] = df.tweet_text.apply(lambda x: process_one(x))

# stem
df['tweet_stem'] = df.tweet_clean.apply(lambda x: stem_one(x))

In [6]:
# transformations so far
df[['label', 'label2', 'tweet_text', 'tweet_clean', 'tweet_stem']].head(10)

,label,label2,tweet_text,tweet_clean,tweet_stem
0,1,0,Warning: penny boards will make you a faggot,Warning penny boards will make you a faggot,Warn penni board will make you faggot
1,2,1,Fuck dykes,Fuck dykes,Fuck dyke
2,2,1,@sizzurp__ @ILIKECATS74 @yoPapi_chulo @brandon...,__HNDL __HNDL __HNDL __HNDL __HNDL at least i ...,least dont look like jefre starr faggot
3,2,1,"""@jayswaggkillah: ""@JacklynAnnn: @jayswaggkill...",__HNDL __HNDL __HNDL Is a fag jackie jealous Nee,fag jacki jealou Nee
4,1,0,@Zhugstubble You heard me bitch but any way I'...,__HNDL You heard me bitch but any way Im back ...,You heard bitch but ani way back texa wtf talk...
5,2,1,@elaynay your a dirty terrorist and your relig...,__HNDL your a dirty terrorist and your religio...,your dirti terrorist and your religion fuck jo...
6,2,1,RT @ivanrabago_: @_WhitePonyJr_ looking like f...,RT __HNDL __HNDL looking like faggots __PUNC_Q...,look like faggot
7,2,1,Well I thought you knew actually RT @KingHorse...,Well I thought you knew actually RT __HNDL Man...,Well thought you knew actual Man whi yall didn...
8,1,0,"@Stonisnipezz I know. It was a joke, faggot.",__HNDL I know It was a joke faggot,know wa joke faggot
9,1,0,I'm tired of people saying I look like my brot...,Im tired of people saying I look like my broth...,tire peopl say look like brother amp call Deon...


In [7]:
# save to file
f_out = '../data/processed/class/classifier-cleaned.csv'

out = df[['label', 'label2', 'tweet_stem']].rename(columns={'label':'y', 'label2':'y2',  'tweet_stem':'X'}, index=None)
out.to_csv(f_out, index=False)